##Importando bibliotecas e base de dados

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

!pip install prefixspan
from prefixspan import PrefixSpan

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for prefixspan: filename=prefixspan-0.5.2-py3-none-any.whl size=11216 sha256=ffa5e0a4e17711a73d5d3d321bec26e0d28163c630f957bf33f8cde4c5d3647b
  Stored in directory: /root/.cache/pip/wheels/95/1a/4a/d02ca86ccf55a25e07bd200a5320b8a584e5e2b75b1168f703
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=b91bf00868f553104e278cdc90261662501a2a95016a0a0d30c639a9edcf6110
  Stored in directory: /root/.cache/pip/wheels/1a/b0/8c/4b75c4116c31f83c8f9f047231251e13cc74481cca4a78a9ce
  Created wheel for extratools: filename=extratools-0.8.2.1-py3-none-any.whl size=28866 sha256=eff6fa92cade8f21d9657d9b7937425eddb3763cdcfc2b9dd1491ed04890cd40
  Stored in directory: /root/.cache/pip/wheels/a8/79/68/6f2edcddbadb15fdcff02e0748a05579d52e78185a4546ab4b
Successfully built prefixspan docopt extratools


In [2]:
events = pd.read_json("https://raw.githubusercontent.com/lucasvitorsr/mineracao-lances-futebol/main/data/processado.json", lines=True)

In [3]:
events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 510336 entries, 0 to 510335
Data columns (total 7 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   minute           510336 non-null  int64 
 1   second           510336 non-null  int64 
 2   period           510336 non-null  int64 
 3   possession       510336 non-null  int64 
 4   possession_team  510336 non-null  object
 5   type             510336 non-null  object
 6   goal             510336 non-null  bool  
dtypes: bool(1), int64(4), object(2)
memory usage: 23.8+ MB


#Preparação final para rodar o Prefixspan

Ainda falta fazer a preparação final dos dados para rodar o modelo, que consistem em agrupar sequências de eventos que resultaram em chutes e filtrar ainda mais as colunas dos dados

In [4]:
events["type"].unique()

array(['Starting XI', 'Half Start', 'Pass', 'Ball Receipt', 'Carry',
       'Clearance', 'Ball Recovery', 'Shot', 'Block', 'Goal Keeper',
       'Miscontrol', 'Pressure', 'Duel', 'Interception', 'Foul Committed',
       'Foul Won', 'Dispossessed', 'Dribble', 'Dribbled Past', 'Error',
       'Injury Stoppage', 'Half End', 'Substitution', 'Bad Behaviour',
       'Referee Ball-Drop', 'Tactical Shift', 'Player Off', 'Player On',
       'Shield', 'Own Goal Against', 'Own Goal For', '50/50', 'Offside',
       'Camera On', 'Camera off'], dtype=object)

- Filtrando os eventos de nosso interesse

In [5]:
events = events[events["type"].isin(['Pass', 'Ball Receipt', 'Carry',
       'Clearance', 'Ball Recovery', 'Shot', 'Block', 'Goal Keeper',
       'Miscontrol', 'Pressure', 'Duel', 'Interception', 'Foul Committed',
       'Foul Won', 'Dispossessed', 'Dribble', 'Dribbled Past', 'Tactical Shift', '50/50', 'Offside'])]

In [6]:
events["type"].unique()

array(['Pass', 'Ball Receipt', 'Carry', 'Clearance', 'Ball Recovery',
       'Shot', 'Block', 'Goal Keeper', 'Miscontrol', 'Pressure', 'Duel',
       'Interception', 'Foul Committed', 'Foul Won', 'Dispossessed',
       'Dribble', 'Dribbled Past', 'Tactical Shift', '50/50', 'Offside'],
      dtype=object)

- Identificando sequências de eventos que terminaram em um chute/shot e que tenham no máximo 75 lances

In [7]:
shots = []
sequence = []
previous_possession = 0
shot = False
for index, row in reversed(list(events.iterrows())): # Sobe no df
    if row["type"] == "Shot":
        if(len(sequence) > 0 and len(sequence) <= 75 and shot):
            shots.append(sequence)
        sequence = []
        sequence.insert(0, row["type"])
        previous_possession = row["possession"]
        shot = True

    elif previous_possession == row["possession"] and shot:
        sequence.insert(0, row["type"])

    else:
        if(len(sequence) > 0 and len(sequence) <= 75 and shot):
            shots.append(sequence)

        sequence = []
        shot = False

if(len(sequence) > 0 and len(sequence) <= 75 and shot):
    shots.append(sequence)

print("Número de sequências: ", len(shots))
for i, seq in enumerate(shots):
    print(f"Sequência {i + 1}: {seq}")

Número de sequências:  3715
Sequência 1: ['Ball Recovery', 'Carry', 'Pass', 'Clearance', 'Ball Recovery', 'Shot']
Sequência 2: ['Shot']
Sequência 3: ['Pass', 'Ball Receipt', 'Pass', 'Ball Receipt', 'Pressure', 'Dribbled Past', 'Dribble', 'Carry', 'Pressure', 'Dribbled Past', 'Dribble', 'Carry', 'Pressure', 'Pass', 'Ball Receipt', 'Carry', 'Pressure', 'Dribbled Past', 'Dribble', 'Carry', 'Shot']
Sequência 4: ['Shot']
Sequência 5: ['Shot']
Sequência 6: ['Pass', 'Ball Receipt', 'Carry', 'Pass', 'Ball Receipt', 'Carry', 'Pass', 'Ball Receipt', 'Carry', 'Pressure', 'Pass', 'Ball Receipt', 'Shot']
Sequência 7: ['Pass', 'Ball Receipt', 'Pass', 'Clearance', 'Ball Recovery', 'Carry', 'Pass', 'Ball Receipt', 'Carry', 'Pass', 'Ball Receipt', 'Carry', 'Pass', 'Ball Receipt', 'Duel', 'Shot']
Sequência 8: ['Interception', 'Carry', 'Pass', 'Ball Receipt', 'Carry', 'Pressure', 'Pass', 'Ball Receipt', 'Carry', 'Pass', 'Ball Receipt', 'Carry', 'Pass', 'Ball Receipt', 'Duel', 'Shot']
Sequência 9: ['Pass'

#Rodando Prefixspan


- suporte de 25%

In [ ]:
lenght = len(shots)
minSuport = 0.25 * lenght
print("Suporte mínimo: ", minSuport)

Suporte mínimo:  928.75


In [ ]:
ps = PrefixSpan(shots)
patterns = ps.frequent(minSuport)
for pattern in patterns:
    print(f"Sequência: {pattern[0]}, Suporte: {pattern[1]}")

Streaming output truncated to the last 5000 lines.
Sequência: 1023, Suporte: ['Ball Receipt', 'Carry', 'Ball Receipt', 'Carry', 'Ball Receipt', 'Carry', 'Carry', 'Carry', 'Pass', 'Shot']
Sequência: 1016, Suporte: ['Ball Receipt', 'Carry', 'Ball Receipt', 'Carry', 'Ball Receipt', 'Carry', 'Carry', 'Carry', 'Ball Receipt']
Sequência: 930, Suporte: ['Ball Receipt', 'Carry', 'Ball Receipt', 'Carry', 'Ball Receipt', 'Carry', 'Carry', 'Carry', 'Ball Receipt', 'Carry']
Sequência: 930, Suporte: ['Ball Receipt', 'Carry', 'Ball Receipt', 'Carry', 'Ball Receipt', 'Carry', 'Carry', 'Carry', 'Ball Receipt', 'Carry', 'Shot']
Sequência: 1016, Suporte: ['Ball Receipt', 'Carry', 'Ball Receipt', 'Carry', 'Ball Receipt', 'Carry', 'Carry', 'Carry', 'Ball Receipt', 'Shot']
Sequência: 970, Suporte: ['Ball Receipt', 'Carry', 'Ball Receipt', 'Carry', 'Ball Receipt', 'Carry', 'Carry', 'Carry', 'Carry']
Sequência: 970, Suporte: ['Ball Receipt', 'Carry', 'Ball Receipt', 'Carry', 'Ball Receipt', 'Carry', 'Carry',

- 35%

In [ ]:
patterns = ps.frequent(0.35 * lenght)
for pattern in patterns:
    print(f"Suporte: {pattern[0]}, Sequência: {pattern[1]}")

Suporte: 1714, Sequência: ['Ball Recovery']
Suporte: 1397, Sequência: ['Ball Recovery', 'Carry']
Suporte: 1397, Sequência: ['Ball Recovery', 'Carry', 'Shot']
Suporte: 1714, Sequência: ['Ball Recovery', 'Shot']
Suporte: 2776, Sequência: ['Carry']
Suporte: 2461, Sequência: ['Carry', 'Pass']
Suporte: 2461, Sequência: ['Carry', 'Pass', 'Shot']
Suporte: 2433, Sequência: ['Carry', 'Pass', 'Ball Receipt']
Suporte: 2258, Sequência: ['Carry', 'Pass', 'Ball Receipt', 'Carry']
Suporte: 1613, Sequência: ['Carry', 'Pass', 'Ball Receipt', 'Carry', 'Pressure']
Suporte: 1323, Sequência: ['Carry', 'Pass', 'Ball Receipt', 'Carry', 'Pressure', 'Carry']
Suporte: 1323, Sequência: ['Carry', 'Pass', 'Ball Receipt', 'Carry', 'Pressure', 'Carry', 'Shot']
Suporte: 1613, Sequência: ['Carry', 'Pass', 'Ball Receipt', 'Carry', 'Pressure', 'Shot']
Suporte: 1357, Sequência: ['Carry', 'Pass', 'Ball Receipt', 'Carry', 'Pressure', 'Ball Receipt']
Suporte: 1357, Sequência: ['Carry', 'Pass', 'Ball Receipt', 'Carry', 'Pres

------

- 45%

In [ ]:
patterns = ps.frequent(0.45 * lenght)
for pattern in patterns:
    print(f"Suporte: {pattern[0]}, Sequência: {pattern[1]}")

Suporte: 1714, Sequência: ['Ball Recovery']
Suporte: 1714, Sequência: ['Ball Recovery', 'Shot']
Suporte: 2776, Sequência: ['Carry']
Suporte: 2461, Sequência: ['Carry', 'Pass']
Suporte: 2461, Sequência: ['Carry', 'Pass', 'Shot']
Suporte: 2433, Sequência: ['Carry', 'Pass', 'Ball Receipt']
Suporte: 2258, Sequência: ['Carry', 'Pass', 'Ball Receipt', 'Carry']
Suporte: 1870, Sequência: ['Carry', 'Pass', 'Ball Receipt', 'Carry', 'Carry']
Suporte: 1870, Sequência: ['Carry', 'Pass', 'Ball Receipt', 'Carry', 'Carry', 'Shot']
Suporte: 2258, Sequência: ['Carry', 'Pass', 'Ball Receipt', 'Carry', 'Shot']
Suporte: 1960, Sequência: ['Carry', 'Pass', 'Ball Receipt', 'Carry', 'Pass']
Suporte: 1932, Sequência: ['Carry', 'Pass', 'Ball Receipt', 'Carry', 'Pass', 'Ball Receipt']
Suporte: 1806, Sequência: ['Carry', 'Pass', 'Ball Receipt', 'Carry', 'Pass', 'Ball Receipt', 'Carry']
Suporte: 1806, Sequência: ['Carry', 'Pass', 'Ball Receipt', 'Carry', 'Pass', 'Ball Receipt', 'Carry', 'Shot']
Suporte: 1932, Sequê

---------------
- 55%

In [ ]:
patterns = ps.frequent(0.55 * lenght)
for pattern in patterns:
    print(f"Suporte: {pattern[0]}, Sequência: {pattern[1]}")

Suporte: 2776, Sequência: ['Carry']
Suporte: 2461, Sequência: ['Carry', 'Pass']
Suporte: 2461, Sequência: ['Carry', 'Pass', 'Shot']
Suporte: 2433, Sequência: ['Carry', 'Pass', 'Ball Receipt']
Suporte: 2258, Sequência: ['Carry', 'Pass', 'Ball Receipt', 'Carry']
Suporte: 2258, Sequência: ['Carry', 'Pass', 'Ball Receipt', 'Carry', 'Shot']
Suporte: 2433, Sequência: ['Carry', 'Pass', 'Ball Receipt', 'Shot']
Suporte: 2277, Sequência: ['Carry', 'Pass', 'Carry']
Suporte: 2277, Sequência: ['Carry', 'Pass', 'Carry', 'Shot']
Suporte: 2776, Sequência: ['Carry', 'Shot']
Suporte: 2340, Sequência: ['Carry', 'Carry']
Suporte: 2340, Sequência: ['Carry', 'Carry', 'Shot']
Suporte: 2436, Sequência: ['Carry', 'Ball Receipt']
Suporte: 2263, Sequência: ['Carry', 'Ball Receipt', 'Carry']
Suporte: 2263, Sequência: ['Carry', 'Ball Receipt', 'Carry', 'Shot']
Suporte: 2436, Sequência: ['Carry', 'Ball Receipt', 'Shot']
Suporte: 3232, Sequência: ['Pass']
Suporte: 3232, Sequência: ['Pass', 'Shot']
Suporte: 3134, Seq

#Análise dos resultados


Com base no resultado do algoritmo Prefixspan, foi possível identificar padrões frequentes em sequências de lances que levam a chutes ao gol. De forma geral, as ações individuais mais frequentes são "Shot" (suporte: 3715), "Pass" (3232) e "Ball Receipt" (3139). Isso mostra que os chutes são o elemento mais comum em sequências finais, enquanto passes e recepções de bola desempenham papéis fundamentais na preparação e transição das jogadas. Além disso, a ação "Carry" (condução da bola) também se destaca como uma das mais frequentes, tanto isoladamente (2776) quanto em combinações como "Carry -> Shot" (2776) e "Carry -> Pass" (2461), evidenciando sua importância na criação de oportunidades.

Algumas sequências identificadas são simples e diretas, como "Carry -> Shot" e "Pass -> Shot", indicando que muitos ataques resultam em chutes após poucas ações. Esse tipo de padrão reflete estratégias rápidas e incisivas, muitas vezes características de contra-ataques ou de momentos em que as defesas adversárias estão desorganizadas. Por outro lado, há sequências mais elaboradas que incluem múltiplas ações antes do chute, como "Pass -> Ball Receipt -> Carry -> Shot" (2591) e "Pass -> Ball Receipt -> Pass -> Shot" (2441). Esses padrões sugerem jogadas construídas, com troca de passes e movimentações coordenadas para abrir espaço na defesa adversária.

Outro ponto relevante é o papel central da ação "Ball Receipt". Sequências como "Pass -> Ball Receipt -> Shot" (3134) e "Ball Receipt -> Carry -> Shot" (2596) mostram que a recepção da bola funciona como um elo crucial entre passes e ações de progressão, como condução ou chute. Essa ação está presente tanto em jogadas rápidas quanto em jogadas mais trabalhadas, evidenciando sua versatilidade no ataque. Além disso, a presença de sequências envolvendo pressão defensiva, como "Pressure -> Shot" (2208), revela que algumas oportunidades de gol surgem mesmo sob condições adversas, como em contra-ataques rápidos ou recuperações de posse no campo adversário.

Comparando os padrões diretos e elaborados, observa-se um equilíbrio interessante. Enquanto as jogadas rápidas, como "Pass -> Shot" e "Carry -> Shot", mostram a eficiência de ataques diretos, as sequências mais longas, como "Pass -> Pass -> Ball Receipt -> Carry -> Shot", refletem estratégias mais complexas, que dependem de maior coordenação e paciência para superar defesas bem posicionadas. Isso sugere que as equipes alternam entre abordagens táticas dependendo da situação do jogo, como o placar ou o tempo restante.

Por fim, a análise evidencia a importância de combinações entre "Carry" e "Pass", que aparecem frequentemente em diferentes contextos. A condução da bola após recepções ou passes parece ser uma estratégia-chave para criar oportunidades de gol. Sequências simples e rápidas podem ser ideais em situações de transição, enquanto padrões mais longos e estruturados são mais utilizados em ataques posicionais. No geral, a flexibilidade das estratégias ofensivas reflete a necessidade de adaptação às condições do jogo.